In [32]:
from surprise import Dataset, evaluate
from surprise import KNNBasic
from surprise import SVD
import pandas as pd
from surprise import accuracy
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

In [33]:
import random
ratings = []
for x in range(227428):
    ratings.append(random.randint(0,1))

In [34]:
df = pd.read_csv("../data/data-ny.csv", sep = ',', header=None, names =  ['userid', 'venueid', 'venuecatid', 'venuecatname','latitude','longitude','timezone','utctime'])

In [62]:
users = pd.DataFrame({'count' : df.groupby(['userid']).size()}).reset_index()
users = users['userid']

In [63]:
venues = pd.DataFrame({'count' : df.groupby(['venueid']).size()}).reset_index()
venues = venues['venueid']

In [70]:
import itertools
user_venuae = list(itertools.product(users, venues))

In [72]:
len(user_venuae)

41514639

In [69]:
df['ratings'] = ratings
df.head()

,userid,venueid,venuecatid,venuecatname,latitude,longitude,timezone,utctime,ratings
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,1
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,0
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,0
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,0
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,1


In [37]:
data = pd.DataFrame()
data['userid'] = df['userid']
data['venueid'] = df['venueid']
data['ratings'] = df['ratings']

In [38]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(df[['userid', 'venueid', 'ratings']], reader)

In [43]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
knn = KNNBasic(sim_options=sim_options)
#algo = SVD()
kf = KFold(n_splits=2)


In [49]:
# #knn.fit(train_data)
# model.fit(train_data)

for trainset, testset in kf.split(train_data):
#     print(trainset)
#     print(testset)
    # train and test algorithm.
    knn.fit(trainset)
    predictions = knn.test(testset)
    print(predictions[:100])
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Computing the cosine similarity matrix...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:277: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
[Prediction(uid=192, iid='4a6a4a50f964a5200bcd1fe3', r_ui=1.0, est=1, details={'actual_k': 2, 'was_impossible': False}), Prediction(uid=402, iid='42829c80f964a5206a221fe3', r_ui=1.0, est=1, details={'actual_k': 39, 'was_impossible': False}), Prediction(uid=59, iid='4f90581ae4b027bf9d4c190b', r_ui=1.0, est=1, details={'actual_k': 33, 'was_impossible': False}), Prediction(uid=195, iid='3fd66200f964a520f6e61ee3', r_ui=0.0, est=1, details={'actual_k': 6, 'was_impossible': False}), Prediction(uid=540, iid='4ef298b729c2e7e6d2b037db', r_ui=0.0, est=1, details={'actual_k': 37, 'was_impossible': False}), Prediction(uid=471, iid='4c13d7d482a3c9b68536fcf8', r_ui=1.0, est=1, details={'actual_k': 34, 'was_impossible': False}), Prediction(uid=388, iid='436e9900f964a520dd291fe3', r_ui=1.0, est=1, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=1029, iid='4ce86f18f1c6236afae35ff0', r_ui=0.0, est=1, details={'actual_k': 35, 'was_imp

Computing the cosine similarity matrix...
Done computing similarity matrix.
[Prediction(uid=22, iid='4abb79b8f964a520cc8320e3', r_ui=1.0, est=1, details={'actual_k': 4, 'was_impossible': False}), Prediction(uid=20, iid='42c9cd80f964a520df251fe3', r_ui=1.0, est=1, details={'actual_k': 24, 'was_impossible': False}), Prediction(uid=344, iid='4bc5b04b41cb76b0a1a03e6f', r_ui=1.0, est=1, details={'actual_k': 36, 'was_impossible': False}), Prediction(uid=777, iid='4c2271ee9085d13a1c5886cc', r_ui=1.0, est=1, details={'actual_k': 32, 'was_impossible': False}), Prediction(uid=448, iid='49eceb82f964a520ad671fe3', r_ui=1.0, est=1, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}), Prediction(uid=70, iid='4b8075bef964a520697430e3', r_ui=1.0, est=1, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}), Prediction(uid=534, iid='4ef377dc6da1a067a58d5a78', r_ui=0.0, est=1, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}), Prediction(uid

In [50]:
len(predictions)

113714

In [55]:
for i in predictions:
    if i.est == 0:
        print(i)